In [2]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.amazon.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.amazon.com/')

True

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm_notebook as tqdm

In [4]:
driver = webdriver.Chrome(executable_path=r'chromedriver.exe')
driver.maximize_window()

In [5]:
driver.get('https://tiki.vn')
time.sleep(.5)
driver.get('https://amazon.com')
driver.get('https://tiki.vn')
time.sleep(.5)
driver.get('https://amazon.com')

In [6]:
elem = driver.find_element_by_id('twotabsearchtextbox')
elem.clear()
elem.send_keys("laptop")
elem.send_keys(Keys.RETURN)
# Press laptop computer on navigation
elem = driver.find_element_by_id('n/565108')
elem = elem.find_element_by_class_name('a-link-normal.s-navigation-item')
elem.click()

In [7]:
# List url
url_list = []

In [8]:
# Get url list in 1 page
def get_url_in_page():
    search_result = driver.find_element_by_class_name('s-result-list.s-search-results.sg-row')
    product_list = search_result.find_elements_by_class_name('a-link-normal.a-text-normal')
    urls = []
    for i in range(0,len(product_list),2):
        product = product_list[i]
        urls.append(product.get_attribute('href'))
    return urls

In [9]:

next_button_container = driver.find_element_by_class_name('a-last')
next_button = next_button_container.find_element_by_tag_name('a')
while next_button is not None:
    next_button.click()
    url_list += get_url_in_page()
    try:
        next_button_container = driver.find_element_by_class_name('a-last')
        next_button = next_button_container.find_element_by_tag_name('a')
    except:
        print('Done')
        break

Done


In [10]:
len(url_list)

9303

In [11]:
with open('links.txt', 'w') as f:
    for url in url_list:
        f.write(url + '\n')

In [12]:
url_list = []
# Read url list from file
with open('links.txt', 'r') as f:
    for line in f.readlines():
        url_list.append(line)

In [13]:
features = ['Screen Size', 'Screen Resolution', 'Processor', 'RAM', 'Hard Drive', 'Graphics Coprocessor', 'Chipset Brand', 'Card Description', 'Number of USB 3.0 Ports','Average Battery Life (in hours)', 'Brand Name', 'Series', 'Operating System', 'Item Weight', 'Product Dimensions', 'Item Dimensions L x W x H', 'Processor Brand', 'Processor Count','Flash Memory Size','Hard Drive Interface', 'Hard Drive Rotational Speed', 'Optical Drive Type','Batteries', 'Date First Available']
features_with_price = features.copy()
features_with_price.append('Price')
features_map = {features_with_price[i]:i for i in range(len(features_with_price))}
features_map['Max Screen Resolution'] = features.index('Screen Resolution')

In [14]:
def getFeatureRows(elems):
    rows = []
    for elem in elems:
        rows += elem.find_elements_by_tag_name('tr')
    return rows

def extractData(rows):
    res = [None]*len(features)
    for row in rows:
        key = row.find_element_by_tag_name('th')
        if key is None or key.text not in features_map:
            continue
        value = row.find_element_by_tag_name('td').text
        res[features_map[key.text]] = value
    return res

In [15]:
res = []
for url in tqdm(url_list):
    time.sleep(.1)
    driver.get(url)
    elems = driver.find_elements_by_class_name('prodDetTable')
    rows = getFeatureRows(elems)
    data = extractData(rows)
    
    price = driver.find_elements_by_id('priceblock_ourprice')
    if price == []:
        price = driver.find_elements_by_id('priceblock_dealprice')
    if price != []:
        price = price[0].text
    else:
        price = None
    data.append(price)
    res.append(data)

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(res, columns=features_with_price)
df.fillna(value=pd.np.nan, inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('laptop.csv')